# 📡 NANSC Intelligent Operations Console (Kaggle Edition)

This notebook is an adaptation of the original `Aero_NAV_Agents.ipynb` designed to run seamlessly in the Kaggle environment. Key modifications include:

1.  **Kaggle Secret Handling**: Uses `UserSecretsClient` to securely access the `GOOGLE_API_KEY`.
2.  **Persistent Storage**: All data (RAG database, session history) is stored in `/kaggle/working/` to persist between sessions.
3.  **Public UI**: Gradio is launched with `share=True` to generate a public link accessible from the Kaggle kernel.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

### Step 1: Install Dependencies

This cell installs all the necessary Python libraries for the agent, including `google-generativeai`, `langchain`, `gradio`, and `chromadb` for the vector store.

In [ ]:
# Installs necessary libraries for Generative AI, RAG, and UI.

%pip install -q -U google-generativeai langchain langchain-community langchain-google-genai chromadb gradio nest_asyncio pypdf pandas duckduckgo-search


### Step 2: Imports and API Key Configuration

Here we import all required modules and configure the Google Generative AI client. It securely fetches the `GOOGLE_API_KEY` from Kaggle secrets.

In [ ]:
# Cell 1: Environment Setup & Imports
# NOTE: If you get import errors, Restart the Runtime/Kernel after this cell runs.

import os
import json
import logging
import asyncio
import nest_asyncio
import pandas as pd
import gradio as gr
from datetime import datetime
from typing import List, Dict, Any
from dataclasses import dataclass

# Third-party imports
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community.tools import DuckDuckGoSearchRun

# Apply asyncio patch for Jupyter/Colab environments
nest_asyncio.apply()

# --- API CONFIGURATION (Kaggle Recommended Practice) ---
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=api_key)
    print("✅ GOOGLE_API_KEY secret loaded from Kaggle.")
except (ImportError, Exception):
    if 'GOOGLE_API_KEY' in os.environ:
        # Fallback for local development if GOOGLE_API_KEY is an environment variable
        genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
        print("✅ GOOGLE_API_KEY loaded from environment variable.")
    else:
        print("⚠️ GOOGLE_API_KEY not found. Please set it as a secret in Kaggle or as an environment variable.")

print("✅ Environment Setup Complete.")

### Step 3: Define Core Services (Layer 1)

This cell sets up the foundational components of our system:
- **SystemConfig**: Defines configuration, including the persistence directory which is now set to `/kaggle/working/nansc_data`.
- **ObservabilityService**: A simple in-memory service for logging events and metrics.
- **SessionManager**: Handles saving and loading chat history to a JSON file.

In [ ]:
# Cell 2: Layer 1 - State, Configuration, and Observability

# --- 1. CONFIGURATION ---
@dataclass
class SystemConfig:
    app_name: str = "NANSC_Intelligent_Console"
    persistence_dir: str = "/kaggle/working/nansc_data"
    model_name: str = "gemini-2.5-flash"

    def __post_init__(self):
        os.makedirs(self.persistence_dir, exist_ok=True)

# --- 2. OBSERVABILITY (Metrics) ---
@dataclass
class TelemetryEvent:
    timestamp: str
    event_type: str
    details: str

class ObservabilityService:
    def __init__(self):
        self.events: List[TelemetryEvent] = []
        self.metrics = {"requests": 0, "tool_usage": 0, "errors": 0}
    
    def log_event(self, event_type: str, details: str):
        event = TelemetryEvent(datetime.now().strftime("%H:%M:%S"), event_type, details)
        self.events.append(event)
        if event_type == "ERROR": self.metrics["errors"] += 1
        elif event_type == "REQUEST": self.metrics["requests"] += 1
        elif event_type == "TOOL_USE": self.metrics["tool_usage"] += 1

    def get_logs(self) -> str:
        return "\n".join([f"[{e.timestamp}] [{e.event_type}] {e.details}" for e in self.events[-15:]])

# --- 3. SESSION MANAGER ---
class SessionManager:
    def __init__(self, config: SystemConfig):
        self.filepath = os.path.join(config.persistence_dir, "sessions.json")
    
    def save_session(self, session_id: str, history: List[Dict]):
        data = {}
        if os.path.exists(self.filepath):
            try:
                with open(self.filepath, 'r') as f: data = json.load(f)
            except: pass
        
        data[session_id] = {"timestamp": datetime.now().isoformat(), "history": history}
        with open(self.filepath, 'w') as f: json.dump(data, f, indent=2)

# Global Instances
sys_config = SystemConfig()
telemetry = ObservabilityService()
session_manager = SessionManager(sys_config)

# Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("NANSC_Core")

print("✅ Layer 1 (State & Observability) Initialized.")

### Step 4: Define Application Logic (Layer 2)

This layer contains the 'brains' of the operation:
- **ICAOTools**: A class containing deterministic tools for domain-specific tasks like airport lookups and address conversions.
- **RAGEngine**: The Retrieval-Augmented Generation engine. It uses `ChromaDB` to store and query PDF documents for context.

In [ ]:
# Cell 3: Layer 2 - Domain Logic, RAG Engine, and Tool Definitions

# --- 1. DOMAIN LOGIC (Deterministic Tools) ---
class ICAOTools:
    AIRPORT_DB = {
        "HECA": "Cairo Intl (Egypt)", "HEBA": "Borg El Arab (Egypt)",
        "OJAA": "Queen Alia (Jordan)", "EGLL": "London Heathrow (UK)",
        "LFPG": "Paris CDG (France)", "KJFK": "JFK New York (USA)"
    }
    
    @staticmethod
    def lookup_airport(icao_code: str) -> str:
        """Looks up an airport location by its 4-letter ICAO code."""
        code = icao_code.upper().strip()
        result = ICAOTools.AIRPORT_DB.get(code, "Unknown ICAO Code")
        telemetry.log_event("TOOL_USE", f"Airport Lookup: {code}")
        return result

    @staticmethod
    def bridge_aftn_to_amhs(aftn_address: str) -> str:
        """Converts legacy AFTN (8-char) to AMHS (X.400) format."""
        addr = aftn_address.upper().strip()
        if len(addr) != 8: return "Error: Address must be 8 chars."
        
        prmd_map = {"HE": "EGYPT", "OJ": "JORDAN", "EG": "UK", "LF": "FRANCE"}
        prefix = addr[:2]
        prmd = prmd_map.get(prefix, "UNKNOWN")
        
        x400 = f"/C=XX/A=ICAO/P={prmd}/O={addr[:4]}/OU1={addr[4:]}"
        telemetry.log_event("TOOL_USE", f"Bridge Conversion: {addr} -> {x400}")
        return x400

    @staticmethod
    def web_search(query: str) -> str:
        """Searches the web for aviation definitions if internal knowledge fails."""
        try:
            search = DuckDuckGoSearchRun()
            res = search.run(query)
            telemetry.log_event("TOOL_USE", f"Web Search: {query}")
            return res
        except:
            return "Search unavailable."

# --- 2. RAG ENGINE ---
class RAGEngine:
    def __init__(self, persistence_dir: str):
        self.persist_dir = os.path.join(persistence_dir, "chroma_db")
        self.embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        self.vector_store = None
        self._init_db()

    def _init_db(self):
        try:
            self.vector_store = Chroma(persist_directory=self.persist_dir, embedding_function=self.embeddings)
        except Exception as e: logger.warning(f"ChromaDB Init Note: {e}")

    def ingest_pdf(self, file_path: str) -> str:
        try:
            loader = PyPDFLoader(file_path)
            docs = loader.load()
            splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
            splits = splitter.split_documents(docs)
            if self.vector_store:
                self.vector_store.add_documents(splits)
                telemetry.log_event("RAG_INGEST", f"Ingested {len(splits)} chunks")
                return f"✅ Ingested {len(splits)} chunks."
            return "❌ Vector Store not ready."
        except Exception as e: return f"Error: {e}"

    def query(self, question: str) -> str:
        if not self.vector_store: return ""
        docs = self.vector_store.similarity_search(question, k=3)
        return "\n".join([d.page_content[:500] for d in docs])

rag_engine = RAGEngine(sys_config.persistence_dir)
# Explicitly list tools for Gemini
tools_list = [ICAOTools.lookup_airport, ICAOTools.bridge_aftn_to_amhs, ICAOTools.web_search]

print("✅ Layer 2 (Knowledge & Tools) Initialized.")

### Step 5: Create Agent and User Interface (Layers 3 & 4)

This is the final step where everything comes together:
- **EnterpriseAgent**: The core agent orchestrator, powered by Gemini, that decides when to use tools, RAG, or its internal knowledge.
- **Gradio Dashboard**: A comprehensive UI that provides a chat interface, batch processing tools, a knowledge base uploader, and a telemetry monitor. The UI is launched with `share=True` to make it accessible.

In [ ]:
# Cell 4: Layer 3 (Agent) & Layer 4 (Robust Dashboard UI)

# --- LAYER 3: AGENT ORCHESTRATOR ---
class EnterpriseAgent:
    def __init__(self):
        # We explicitly list tools here to ensure Gemini knows about them
        self.model = genai.GenerativeModel(
            model_name=sys_config.model_name,
            tools=tools_list, 
            system_instruction="""
            You are the NANSC Intelligent Operations Console Assistant.
            
            OPERATIONAL PROTOCOL:
            1. DEFINITIONS: If the user asks "What is...", answer from your internal knowledge. 
               If unsure, use the 'web_search' tool.
            2. CODES: If an ICAO code (4 letters) or AFTN address (8 letters) is detected, 
               ALWAYS use 'lookup_airport' or 'bridge_aftn_to_amhs' tools automatically.
            3. PROCEDURES: If asked about rules/regs, refer to the RAG Context provided.
            
            Be professional, concise, and helpful.
            """
        )
        self.chat = self.model.start_chat(enable_automatic_function_calling=True)

    async def process_message(self, message: str) -> str:
        """
        Async handler to prevent 'Task attached to different loop' errors.
        """
        # 1. RAG Context Injection (Simple Keyword Check)
        rag_context = ""
        if any(x in message.lower() for x in ["procedure", "rule", "reg", "manual", "doc"]):
            rag_context = rag_engine.query(message)
            if rag_context: 
                message = f"Reference Info from Manuals:\n{rag_context}\n\nUser Question: {message}"
        
        try:
            # 2. Async Generation
            response = await self.chat.send_message_async(message)
            telemetry.log_event("REQUEST", "Message processed successfully")
            
            # 3. Session Persistence
            hist_serialized = [{"role": p.role, "parts": [pt.text for pt in p.parts]} for p in self.chat.history]
            session_manager.save_session("web_user", hist_serialized)
            
            return response.text
        except Exception as e:
            telemetry.log_event("ERROR", str(e))
            return f"⚠️ System Error: {str(e)}"

# --- LAYER 4: DASHBOARD UI ---
agent = EnterpriseAgent()

# 1. Chat Wrapper (Async)
async def chat_wrapper(message, history):
    if not message: return ""
    return await agent.process_message(message)

# 2. Tool Wrappers
def batch_tool_wrapper(text_input, operation):
    lines = [l.strip() for l in text_input.split('\n') if l.strip()]
    results = []
    for line in lines:
        if operation == "Convert AFTN":
            res = ICAOTools.bridge_aftn_to_amhs(line)
        else:
            res = ICAOTools.lookup_airport(line)
        results.append({"Input": line, "Result": res})
    return pd.DataFrame(results)

def ingest_wrapper(files):
    if not files: return "No files provided."
    return "\n".join([rag_engine.ingest_pdf(f.name) for f in files])

def get_stats_wrapper():
    return json.dumps(telemetry.metrics, indent=2), telemetry.get_logs()

# 3. Layout Construction (Removed 'theme' arg for compatibility)
with gr.Blocks(title="NANSC Ops Console") as demo:
    
    # Header Section
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                # 📡 NANSC Intelligent Operations Console
                **Civil Aviation Telecommunications | AI-Powered Assistant**
                """
            )
    
    # Main Dashboard Layout
    with gr.Row():
        
        # LEFT COLUMN: Tools & Admin (Sidebar style)
        with gr.Column(scale=1, min_width=300):
            
            with gr.Accordion("🛠️ Batch Tools", open=True):
                gr.Markdown("Process multiple items at once.")
                b_in = gr.TextArea(lines=3, placeholder="HECAYFYX\nOJAA\nEGLL", show_label=False)
                b_op = gr.Radio(["Convert AFTN", "Lookup Airport"], value="Convert AFTN", show_label=False)
                b_btn = gr.Button("🚀 Run Batch", variant="primary")
                b_out = gr.Dataframe(headers=["Input", "Result"], wrap=True)
                b_btn.click(batch_tool_wrapper, inputs=[b_in, b_op], outputs=b_out)

            with gr.Accordion("📚 Knowledge Base", open=False):
                gr.Markdown("Upload PDFs to RAG Engine.")
                f_up = gr.File(file_count="multiple", file_types=[".pdf"])
                up_btn = gr.Button("📥 Ingest")
                up_log = gr.Textbox(show_label=False, placeholder="Status...")
                up_btn.click(ingest_wrapper, inputs=[f_up], outputs=[up_log])
                
            with gr.Accordion("⚙️ Telemetry", open=False):
                stat_btn = gr.Button("🔄 Refresh")
                stat_json = gr.Code(language="json", label="Metrics", lines=5)
                stat_log = gr.TextArea(label="System Logs", lines=5)
                stat_btn.click(get_stats_wrapper, outputs=[stat_json, stat_log])

        # RIGHT COLUMN: Chat Interface
        with gr.Column(scale=3):
            gr.ChatInterface(
                fn=chat_wrapper,
                examples=[
                    "What is AMHS?",
                    "Convert HECAYFYX to X.400",
                    "Where is OJAA?",
                    "Lookup EGLL"
                ],
                title="Operations Assistant",
                description="Interact with the Enterprise Agent. Ask about definitions, convert addresses, or query manuals."
            )

print("🚀 Launching NANSC Console...")
demo.launch(share=True, debug=True)

### Author

*[Your Name Here]*